Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Xin Qu"
COLLABORATORS = ""

---

# Integrity constraints with SQL
This question is almost identical to second question of datalog assignment. We will find integrity constraint violations in `publications` dataset but using SQL now.

In [2]:
%reload_ext sql
%reload_ext lib.sqlite.sqlite_evaluate_magic
import os

### Connect to the database

In [3]:
# Connect to database. Following command will connect you to the database.
# Any query that you will run after this cell will be run on the `publications` database.
# You can always change the path in database url and connect to some other sqlite db of your choice.
publications_db_url = 'sqlite:///' + os.path.expanduser('~/data_readonly/sqlite/databases/publications.db')
%sql $publications_db_url

'Connected: @/home/jovyan/data_readonly/sqlite/databases/publications.db'

The database has two tables: (1) Publication (2) Cites  
Header of output of following two queries will tell you the column names in these tables.

In [4]:
%%sql 
select * from Publication limit 1;

 * @/home/jovyan/data_readonly/sqlite/databases/publications.db
 * sqlite:////home/jovyan/data_readonly/sqlite/databases/publications.db
Done.


pid,authors,year,title,journal,vol,no,fp,lp,publisher
6755,hyatt,1872,fossil,bullmcz,5,5,91,9,publisher1


In [5]:
%%sql 
select * from Cites limit 1;

 * @/home/jovyan/data_readonly/sqlite/databases/publications.db
 * sqlite:////home/jovyan/data_readonly/sqlite/databases/publications.db
Done.


citing,cited
4711,2020


## We will now write various queries to find "bad" (inconsistent) data. 
If output format is not clear from wording of the question, look at expected output and make your query return in same schema.

### [12 points] The key attribute ID should uniquely determine all other attributes.

In DENIAL form we report all IC violations, i.e., where there are at least two rows having the same ID, but some differing attributes.


In [6]:
%%sql
Problem2a_FD_1 <<
-- Your query here. Don't change variable name.
select p1.pid, p1.authors, p1.year, p1.title, p1.journal, p1.vol, p1.no, p1.fp, p1.lp, p1.publisher
from publication as p1, publication as p2 where p1.pid == p2.pid and p1.authors != p2.authors


 * @/home/jovyan/data_readonly/sqlite/databases/publications.db
 * sqlite:////home/jovyan/data_readonly/sqlite/databases/publications.db
Done.
Saving data to local variable Problem2a_FD_1['result']
Saving query to local variable Problem2a_FD_1['query']


pid,authors,year,title,journal,vol,no,fp,lp,publisher
4407,kummel,1969,ammonoids,bullmcz,137,3,476,None,publisher2
4407,doe,2015,foobar,bullmcz,10,1,10,1,None


In [7]:
# Run this cell to see expected output of previous query
%sql_expected_output Problem2a_FD_1

pid,authors,year,title,journal,vol,no,fp,lp,publisher
4407,kummel,1969,ammonoids,bullmcz,137,3,476,None,publisher2
4407,doe,2015,foobar,bullmcz,10,1,10,1,None


In [8]:
# We evaluate your query to see if it gave expected output.
%sql_evaluate Problem2a_FD_1

## [11 points] Every journal has a single publisher, i.e., Journal --> Publisher
In denial mode, we report the journals which have multiple publishers, two publishers at a time.

In [9]:
%%sql
Problem2a_FD_2 <<
-- Your query here. Don't change variable name.
select t.journal, t.publisher from (select distinct p1.publisher, p1.journal
from publication as p1, publication as p2 where p1.journal == p2.journal
and (p1.publisher != p2.publisher or (p1.publisher is null and p2.publisher is not null) 
     or (p1.publisher is not null and p2.publisher is null))) as t;


 * @/home/jovyan/data_readonly/sqlite/databases/publications.db
 * sqlite:////home/jovyan/data_readonly/sqlite/databases/publications.db
Done.
Saving data to local variable Problem2a_FD_2['result']
Saving query to local variable Problem2a_FD_2['query']


journal,publisher
bullmcz,publisher1
bullmcz,publisher2
bullmcz,None


In [10]:
# Run this cell to see expected output of previous query
%sql_expected_output Problem2a_FD_2

journal,publisher
bullmcz,publisher1
bullmcz,publisher2
bullmcz,None


In [11]:
# We evaluate your query to see if it gave expected output.
%sql_evaluate Problem2a_FD_2

### [11 points] The last page cannot be smaller than the first page.
In DENIAL form, we report the ones for which last page is smaller than first.

In [12]:
%%sql
Problem2a_NC_1 <<
-- Your query here. Don't change variable name.
select distinct p1.pid, p1.authors, p1.year, p1.title, p1.journal, p1.vol, p1.no, p1.fp, p1.lp, p1.publisher
from publication as p1, publication as p2 where p1.pid == p2.pid and p1.lp < p2.fp;


 * @/home/jovyan/data_readonly/sqlite/databases/publications.db
 * sqlite:////home/jovyan/data_readonly/sqlite/databases/publications.db
Done.
Saving data to local variable Problem2a_NC_1['result']
Saving query to local variable Problem2a_NC_1['query']


pid,authors,year,title,journal,vol,no,fp,lp,publisher
6755,hyatt,1872,fossil,bullmcz,5,5,91,9,publisher1
4407,doe,2015,foobar,bullmcz,10,1,10,1,None


In [13]:
# Run this cell to see expected output of previous query
%sql_expected_output Problem2a_NC_1

pid,authors,year,title,journal,vol,no,fp,lp,publisher
6755,hyatt,1872,fossil,bullmcz,5,5,91,9,publisher1
4407,doe,2015,foobar,bullmcz,10,1,10,1,None


In [14]:
# We evaluate your query to see if it gave expected output.
%sql_evaluate Problem2a_NC_1

### [11 points] Inclusion Dependency: Every cited publication in CITES also occurs in PUBLICATION.

In DENIAL form, we report those publications which are in CITES but not in PUBLICATION.

In [15]:
%%sql
Problem2b_ID <<
-- Your query here. Don't change variable name.
select cited from cites where cited not in (select pid from publication);


 * @/home/jovyan/data_readonly/sqlite/databases/publications.db
 * sqlite:////home/jovyan/data_readonly/sqlite/databases/publications.db
Done.
Saving data to local variable Problem2b_ID['result']
Saving query to local variable Problem2b_ID['query']


cited
2020
3799


In [16]:
# Run this cell to see expected output of previous query
%sql_expected_output Problem2b_ID

cited_but_not_in_Publication
2020
3799


In [17]:
# We evaluate your query to see if it gave expected output.
%sql_evaluate Problem2b_ID

### [11 points] If P1 cites P2 then P2's year of publication cannot be greater than P1.

In [18]:
%%sql
Problem2b_NC_2 <<
-- Your query here. Don't change variable name.
select citing, cited, p1.year, p2.year from cites as c
join publication as p1 on p1.pid == c.citing
join publication as p2 on p2.pid == c.cited
where p1.year < p2.year


 * @/home/jovyan/data_readonly/sqlite/databases/publications.db
 * sqlite:////home/jovyan/data_readonly/sqlite/databases/publications.db
Done.
Saving data to local variable Problem2b_NC_2['result']
Saving query to local variable Problem2b_NC_2['query']


citing,cited,year,year_1
2044,2580,1934,1962


In [19]:
# Run this cell to see expected output of previous query
%sql_expected_output Problem2b_NC_2

citing,cited,citing_pub_year,cited_pub_year
2044,2580,1934,1962


In [20]:
# We evaluate your query to see if it gave expected output.
%sql_evaluate Problem2b_NC_2